# 登录

In [1]:
import requests
import json
from os.path import expanduser
from requests.auth import HTTPBasicAuth
def sign_in():
    with open(expanduser('brain_credentials.txt')) as f:
        credentials = json.load(f)
    username, password = credentials
    sess = requests.Session()
    sess.auth = HTTPBasicAuth(username, password)
    response = sess.post('https://api.worldquantbrain.com/authentication')
    print(requests.status_codes)
    print(response.json())
    return sess

sess = sign_in()


<module 'requests.status_codes' from 'c:\\Users\\wyq\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\requests\\status_codes.py'>
{'user': {'id': 'YW10763'}, 'token': {'expiry': 14400.0}, 'permissions': ['CONSULTANT', 'MULTI_SIMULATION', 'PROD_ALPHAS', 'REFERRAL', 'VISUALIZATION', 'WORKDAY']}


## 获取数据集ID为fundamental6(Company Fundamental Data for Equity)下的所有数据字段


In [2]:
def get_datafields(
    s,
    searchScope,
    dataset_id: str = '',
    search: str = ''
):
    import pandas as pd
    instrument_type = searchScope['instructmentType']
    region = searchScope['region']
    delay = searchScope['delay']
    universe = searchScope['universe']
    if len(search) == 0:
        url_template = "https://api.worldquantbrain.com/data-fields?" +\
            f"&instrumentType={instrument_type}" +\
            f"&region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50" +\
            "&offset={x}"
        count = s.get(url_template.format(x=0)).json()['count'] 
    else:
        url_template = "https://api.worldquantbrain.com/data-fields?" +\
            f"&instructmentType={instrument_type}" +\
            f"&region={region}&delay={str(delay)}&universe={universe}&limit=50" +\
            f"&search={search}" +\
            "&offset={x}"
        count = 100

    datafields_list=[]
    for x in range(0, count, 50):
        datafields = s.get(url_template.format(x=x))
        datafields_list.append(datafields.json()['results'])
    datafields_list_flat = [item for sublist in datafields_list for item in sublist]

    datafields_df = pd.DataFrame(datafields_list_flat)
    return datafields_df

In [3]:
searchScope = {'region': 'USA', 'delay':'1', 'universe':'TOP3000', 'instructmentType':'EQUITY'}
fnd6 = get_datafields(s=sess,searchScope=searchScope,dataset_id='fundamental6')
fnd6 = fnd6[fnd6['type']=="MATRIX"]

In [4]:
datafields_list_fnd6 = fnd6['id'].values
datafields_list_fnd6

array(['assets', 'assets_curr', 'bookvalue_ps', 'capex', 'cash',
       'cash_st', 'cashflow', 'cashflow_dividends', 'cashflow_fin',
       'cashflow_invst', 'cashflow_op', 'cogs', 'current_ratio', 'debt',
       'debt_lt', 'debt_st', 'depre_amort', 'ebit', 'ebitda', 'employee',
       'enterprise_value', 'eps', 'equity', 'fnd6_acdo', 'fnd6_acodo',
       'fnd6_acox', 'fnd6_acqgdwl', 'fnd6_acqintan',
       'fnd6_adesinda_curcd', 'fnd6_aldo', 'fnd6_am', 'fnd6_aodo',
       'fnd6_aox', 'fnd6_aqc', 'fnd6_aqi', 'fnd6_aqs', 'fnd6_beta',
       'fnd6_capxv', 'fnd6_ceql', 'fnd6_ch', 'fnd6_ci', 'fnd6_cibegni',
       'fnd6_cicurr', 'fnd6_cidergl', 'fnd6_cik', 'fnd6_cimii',
       'fnd6_ciother', 'fnd6_cipen', 'fnd6_cisecgl', 'fnd6_citotal',
       'fnd6_city', 'fnd6_cld2', 'fnd6_cld3', 'fnd6_cld4', 'fnd6_cld5',
       'fnd6_cptmfmq_actq', 'fnd6_cptmfmq_atq', 'fnd6_cptmfmq_ceqq',
       'fnd6_cptmfmq_dlttq', 'fnd6_cptmfmq_dpq', 'fnd6_cptmfmq_lctq',
       'fnd6_cptmfmq_oibdpq', 'fnd6_cptmfmq_o

### 将datafields替换到Alpha模板(框架)中, 这个模板在这边可以是group_rank(ts_rank({fundamental model data}, 252), industy), 然后批量生成Alpha

In [5]:
group_compare_op = ['group_rank', 'group_zscore', 'group_neutralize']
ts_compare_op = ['ts_rank', 'ts_zscore', 'ts_av_diff']
company_fundmentals = datafields_list_fnd6
days = [60,200]
group = ['market', 'industry', 'subindustry', 'sector', 'densify(pv13_h_f1_sector)']

alpha_expressions = []

for gco in group_compare_op:
    for tco in ts_compare_op:
        for cf in company_fundmentals:
            for d in days:
                for grp in group:
                    alpha_expressions.append(f"{gco}({tco}({cf},{d}),{grp})")
                        
print(f'there are total {len(alpha_expressions)} alpha expressions')

there are total 51660 alpha expressions


In [6]:
alpha_expressions[:20]

['group_rank(ts_rank(assets,60),market)',
 'group_rank(ts_rank(assets,60),industry)',
 'group_rank(ts_rank(assets,60),subindustry)',
 'group_rank(ts_rank(assets,60),sector)',
 'group_rank(ts_rank(assets,60),densify(pv13_h_f1_sector))',
 'group_rank(ts_rank(assets,200),market)',
 'group_rank(ts_rank(assets,200),industry)',
 'group_rank(ts_rank(assets,200),subindustry)',
 'group_rank(ts_rank(assets,200),sector)',
 'group_rank(ts_rank(assets,200),densify(pv13_h_f1_sector))',
 'group_rank(ts_rank(assets_curr,60),market)',
 'group_rank(ts_rank(assets_curr,60),industry)',
 'group_rank(ts_rank(assets_curr,60),subindustry)',
 'group_rank(ts_rank(assets_curr,60),sector)',
 'group_rank(ts_rank(assets_curr,60),densify(pv13_h_f1_sector))',
 'group_rank(ts_rank(assets_curr,200),market)',
 'group_rank(ts_rank(assets_curr,200),industry)',
 'group_rank(ts_rank(assets_curr,200),subindustry)',
 'group_rank(ts_rank(assets_curr,200),sector)',
 'group_rank(ts_rank(assets_curr,200),densify(pv13_h_f1_sector)

In [7]:
alpha_list = []

for alpha_expression in alpha_expressions:
    simulation_data = {
    'type': 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'USA',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 0,
        'neutralization': 'SUBINDUSTRY',
        'truncation': 0.01,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'OFF',
        'language': 'FASTEXPR',
        'visualization': False,
            },
'regular': alpha_expression
}
    alpha_list.append(simulation_data)

In [8]:
alpha_list[0]

{'type': 'REGULAR',
 'settings': {'instrumentType': 'EQUITY',
  'region': 'USA',
  'universe': 'TOP3000',
  'delay': 1,
  'decay': 0,
  'neutralization': 'SUBINDUSTRY',
  'truncation': 0.01,
  'pasteurization': 'ON',
  'unitHandling': 'VERIFY',
  'nanHandling': 'OFF',
  'language': 'FASTEXPR',
  'visualization': False},
 'regular': 'group_rank(ts_rank(assets,60),market)'}

In [9]:
len(alpha_list)

51660

## 将Alpha一个个的发送至服务器进行回测, 并检查是否断线
### 设置log

In [10]:
import logging
logging.basicConfig(filename='simulation.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
from time import sleep
import logging

alpha_fail_attempt_tolerance = 15 # 每个alpha允许最大失败尝试次数

for alpha in alpha_list[5:]:
    keep_trying = True
    failure_count = 0

    while keep_trying:
        try:
            sim_resp = sess.post(
                'https://api.worldquantbrain.com/simulations',
                json=alpha
            )
            sim_progress_url = sim_resp.headers['Location']
            logging.info(f'位置是{sim_progress_url}')
            print(f'位置是{sim_progress_url}')
            keep_trying = False
        except Exception as e:
            logging.error(f'无位置, 休息5s后重试, 错误: {str(e)}')
            print("无位置, 休息5s后重试")
            sleep(5)
            failure_count += 1
            if failure_count >= alpha_fail_attempt_tolerance:
                sess = sign_in()
                failure_count = 0
                logging.error(f"尝试次数过多, 无法获取alpha位置, 移动到下一个alpha{alpha['regular']}")
                print(f"尝试次数过多, 无法获取alpha位置, 移动到下一个alpha{alpha['regular']}")
                break

位置是https://api.worldquantbrain.com/simulations/1yJoMxgM4J9aovXQ6eoqv1
位置是https://api.worldquantbrain.com/simulations/2KS9Q2BZ57G92Nkw2sZM5g
位置是https://api.worldquantbrain.com/simulations/2IBvME2024ORc3GaWNmowbW
位置是https://api.worldquantbrain.com/simulations/3R65W1gM958FaZHJZoTAAl3
位置是https://api.worldquantbrain.com/simulations/4qV9vKqU4SUbeK185hetXcT
位置是https://api.worldquantbrain.com/simulations/1JDMBL6sP56J8MOkYmhdSdn
位置是https://api.worldquantbrain.com/simulations/1v9rKw7w14Y6bMHWEcO1mQh
无位置, 休息15s后重试
位置是https://api.worldquantbrain.com/simulations/2t26s64Bh4zmctckJPZglYm
位置是https://api.worldquantbrain.com/simulations/3YAiYjgdt53DcnF15UCczFO0
位置是https://api.worldquantbrain.com/simulations/IfsT89Kn4j3bTCmA0JYDcq
无位置, 休息15s后重试
无位置, 休息15s后重试
位置是https://api.worldquantbrain.com/simulations/1lXU7i9Do4ZJcKJV57GMs8q
位置是https://api.worldquantbrain.com/simulations/2gFSqc7h34SOcjgAGFpKdT9
位置是https://api.worldquantbrain.com/simulations/2Z4q2a1fy4K4anC1d1qT0cRc
位置是https://api.worldquantbrain.com/s

KeyboardInterrupt: 